In [1]:
import re
import requests
import pandas as pd

URL = "https://www.richbourse.com/common/variation/index/veille/tout"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36",
    "Accept-Language": "fr-FR,fr;q=0.9,en;q=0.8",
}

def _to_number(x: str | float | int | None):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    s = str(x).replace("\xa0", "").replace(" ", "").replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    if s in ("", "."):
        return None
    return float(s)

# 1) Récupérer la page avec un User-Agent
resp = requests.get(URL, headers=HEADERS, timeout=20)
resp.raise_for_status()
html = resp.text

# 2) Lire le tableau
tables = pd.read_html(html, decimal=",", thousands=" ")
if not tables:
    raise RuntimeError("Aucun tableau trouvé.")

df = tables[0].copy()

# 3) Repérer les colonnes (tolérant aux variations d'intitulés)
norm = {c.lower().strip(): c for c in df.columns}
def col_like(options):
    for k, orig in norm.items():
        for opt in options:
            if opt in k:
                return orig
    raise KeyError(f"Colonne introuvable: {options}")

col_symbole = col_like(["symbole"])
col_cours   = col_like(["cours actuel", "coursactuel"])
col_capi    = col_like(["capitalisation"])

# 4) Sélection / nettoyage
work = df[[col_symbole, col_cours, col_capi]].rename(
    columns={col_symbole: "Ticker", col_cours: "Cours", col_capi: "Capitalisation"}
)

work["Cours"] = work["Cours"].map(_to_number)
work["Capitalisation"] = work["Capitalisation"].map(_to_number)

# 5) Calcul nb_actions_actuel = capitalisation / cours
work["nb_actions_actuel"] = (work["Capitalisation"] / work["Cours"]).round().astype("Int64")

out = work[["Ticker", "nb_actions_actuel"]].dropna()
out["Ticker"] = out["Ticker"].astype(str).str.strip()

# 6) Export
out.to_csv("richbourse_nb_actions_actuel.csv", index=False, encoding="utf-8")
print("✅ Export : richbourse_nb_actions_actuel.csv")
print(out.head(10))


✅ Export : richbourse_nb_actions_actuel.csv
  Ticker  nb_actions_actuel
0   ABJC           10912000
1   BICB           57759800
2   BICC           16666670
3   BNBC            6624000
4   BOAB           40561048
5  BOABF           44000000
6   BOAC           40000000
7   BOAM           27450000
8   BOAN           20800000
9   BOAS           36000000


/var/folders/0n/svq7n_mn2lg1fv5jbk66hzg00000gn/T/ipykernel_2671/3617161852.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, decimal=",", thousands=" ")
